In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from collections import Counter
from torch.optim import Adam
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os

## Dataset

In [10]:
human_faces_dir = '/kaggle/input/human-faces/Humans'
flowers_dir = '/kaggle/input/face-dataset/human-swap/'

human_faces_files = len(os.listdir(human_faces_dir))
flowers_files = len(os.listdir(flowers_dir))

print(f"Number of files in human_faces_dir: {human_faces_files}")
print(f"Number of files in flowers_dir: {flowers_files}")

Number of files in human_faces_dir: 7219
Number of files in flowers_dir: 6676


In [11]:
my_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

class CustomDataset(Dataset):
    def __init__(self, human_faces_dir, flowers_dir, transform=None):
        self.human_faces_dir = human_faces_dir
        self.flowers_dir = flowers_dir
        
        human_faces_images = [os.path.join(human_faces_dir, fname) for fname in os.listdir(human_faces_dir)]
        self.human_faces_images = random.sample(human_faces_images, 300)
        flowers_images = [os.path.join(flowers_dir, fname) for fname in os.listdir(flowers_dir)]
        self.flowers_images = random.sample(flowers_images, 200)
        
        self.all_images = self.human_faces_images + self.flowers_images
        self.labels = [1] * len(self.human_faces_images) + [0] * len(self.flowers_images)
        self.transform = transform

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path = self.all_images[idx]
        image = Image.open(img_path).convert("RGB")
        
        label = self.labels[idx]
        
        if self.transform: image = self.transform(image)
        return image, label

dataset = CustomDataset(human_faces_dir, flowers_dir, transform=my_transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
for images, labels in dataloader:
    print(images.shape)
    print(labels)
    break

torch.Size([128, 3, 128, 128])
tensor([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 0, 1, 1])


In [13]:
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        test_images = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('.jpg', '.png', '.jpeg'))]
        test_images = random.sample(test_images, 128)
        self.all_images = test_images
        self.transform = transform

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path = self.all_images[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform: image = self.transform(image)
        return image

test_dir = "/kaggle/input/face-dataset/human-swap/"
test_dataset = TestDataset(test_dir, transform=my_transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [14]:
for images in test_dataloader:
    print(images.shape)
    break

torch.Size([128, 3, 128, 128])


## CNN

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 256, kernel_size=9, stride=1, padding=4)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=9, stride=2, padding=4)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=9, stride=2, padding=4)
        self.conv4 = nn.Conv2d(64, 32, kernel_size=9, stride=2, padding=4)
        self.fc_input_size = self._get_fc_input_size()
        self.fc = nn.Linear(self.fc_input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def _get_fc_input_size(self):
        dummy_input = torch.zeros(1, 3, 128, 128)
        x = self.conv1(dummy_input)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        return x.size(1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

    def loss(self, output, target):
        target = target.unsqueeze(1)
        target = target.float()
        return F.binary_cross_entropy(output, target)


In [16]:
cnn = CNN().to(device)
optimizer = Adam(cnn.parameters())

n_epochs = 20
for epoch in range(n_epochs):
    cnn.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    for batch_id, (data, target) in enumerate(tqdm(dataloader)):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = cnn(data)
        loss = cnn.loss(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        preds = (output >= 0.5).float()
        correct_train += preds.eq(target.view_as(preds)).sum().item()
        total_train += target.size(0)
    
    train_accuracy = correct_train / total_train
    avg_train_loss = train_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_train_loss:.4f} - Accuracy: {train_accuracy:.4f}")

100%|██████████| 4/4 [00:16<00:00,  4.13s/it]


Epoch 1/20 - Loss: 1.3310 - Accuracy: 0.5200


100%|██████████| 4/4 [00:12<00:00,  3.15s/it]


Epoch 2/20 - Loss: 0.6982 - Accuracy: 0.4000


100%|██████████| 4/4 [00:12<00:00,  3.07s/it]


...

Epoch 18/20 - Loss: 0.6731 - Accuracy: 0.6000


100%|██████████| 4/4 [00:12<00:00,  3.08s/it]


Epoch 19/20 - Loss: 0.6728 - Accuracy: 0.6000


100%|██████████| 4/4 [00:12<00:00,  3.05s/it]

Epoch 20/20 - Loss: 0.6730 - Accuracy: 0.6000


In [17]:
cnn.eval()
test_loss = 0
correct_test = 0
total_test = 0

for data in test_dataloader:
    data = data.to(device)
    output = cnn(data)
    target = torch.zeros(output.size(0)).float()
    
    target = target.to(device)
    loss = cnn.loss(output, target)

    test_loss += loss.item()

    preds = (output >= 0.5).float()
    correct_test += preds.eq(target.view_as(preds)).sum().item()
    total_test += target.size(0)

test_accuracy = correct_test / total_test
avg_test_loss = test_loss / len(test_dataloader)
print(f"Loss: {avg_test_loss:.4f} - Accuracy: {test_accuracy:.4f}")

Loss: 0.9030 - Accuracy: 0.0000
